In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DataConversionWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [2]:
train_df = pd.read_csv('data/train.csv')

train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
test_df = pd.read_csv('data/test.csv')

test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [6]:
# title extraction from Name column

def extract_title(df):
    df['Title'] = df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

    df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
    df['Title'] = df['Title'].replace(['Countess', 'Mme', 'Lady'], 'Mrs')

    other_male = ['Col', 'Capt', 'Don', 'Jonkheer', 'Major', 'Rev', 'Sir', 'Dr']
    df['Title'] = df['Title'].replace(other_male, 'Other Male')

    df.loc[(df['Sex'] == 'female') & (df['Title'] == 'Other Male'), 'Title'] = 'Mrs'

    df['Title'] = df['Title'].map({'Mrs': 0, 'Miss': 1, 'Mr': 2, 'Master': 3, 'Other Male': 4})

    df.loc[(df['Title'].isnull()) & (df['Sex'] == 'female'), 'Title'] = 0
    df.loc[(df['Title'].isnull()) & (df['Sex'] == 'male'), 'Title'] = 2
    
    return df

In [7]:
# age feature engineering

def infer_age(df):
    age_inference = np.zeros((3,5))
    pclasses = [1, 2, 3]
    titles = [0, 1, 2, 3, 4]

    for pclass in pclasses:
        for title in titles:
            filtered_df = df[(df['Pclass'] == pclass) & (df['Title'] == title)]

            median = filtered_df['Age'].median()
            if np.isnan(median):
                median = df[df['Pclass'] == pclass]['Age'].median()

            age_inference[pclass - 1, title] = int(median)

    for idx1, pclass in enumerate(pclasses):
        for idx2, title in enumerate(titles):
            df.loc[(df['Age'].isnull()) & (df['Sex'] == idx1) & (df['Pclass'] == idx2 + 1), 'Age'] = age_inference[idx1,idx2]

    return df

In [8]:
most_common_city = train_df['Embarked'].value_counts().idxmax()

def data_preparation(df):
    df = extract_title(df)

    # drop columns that won't be useful during classification
    columns_to_drop = ['Survived', 'Name', 'Ticket', 'Cabin', 'PassengerId']
    
    for column in columns_to_drop:
        if column in df.columns:
            df= df.drop(column, axis='columns')

    df.loc[:,'Sex'] = df.loc[:,'Sex'].map({'male': 0,'female': 1})

    # Embarked related operations
    df['Embarked'] = df['Embarked'].fillna(most_common_city) # fill Nan values in Embarked column with the most common embark city
    df['Embarked'] = df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

    # Age related operations
    df = infer_age(df) # infer age for missing data
    df['Age'] = pd.qcut(df['Age'], 5, labels=[0, 1, 2, 3, 4]) # map Age to 5 equal-sized columns

    # Fare related operations
    median_fare = df['Fare'].median() 
    df['Fare'] = df['Fare'].fillna(median_fare)
    df['Fare'] = pd.qcut(df['Fare'], 5, labels=[0, 1, 2, 3, 4]) # map Fare to 5 equal-sized columns

    df['Age'] = df['Age'].astype('int64')
    df['Fare'] = df['Fare'].astype('int64')
    
    return df

In [9]:
x_train, y_train = data_preparation(train_df), train_df[['Survived']]

print(x_train.shape)
print(y_train.shape)

x_train

(891, 8)
(891, 1)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,3,0,1,1,0,0,2,2
1,1,1,3,1,0,4,0,0
2,3,1,1,0,0,1,2,1
3,1,1,3,1,0,4,2,0
4,3,0,3,0,0,1,2,2
...,...,...,...,...,...,...,...,...
886,2,0,1,0,0,2,2,4
887,1,1,0,0,0,3,2,1
888,3,1,2,1,2,3,2,1
889,1,0,1,0,0,3,0,2


In [10]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Pclass    891 non-null    int64 
 1   Sex       891 non-null    object
 2   Age       891 non-null    int64 
 3   SibSp     891 non-null    int64 
 4   Parch     891 non-null    int64 
 5   Fare      891 non-null    int64 
 6   Embarked  891 non-null    int64 
 7   Title     891 non-null    int64 
dtypes: int64(7), object(1)
memory usage: 55.8+ KB


### Grid Search for the best Random Forest Classifier

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings("ignore", category=UserWarning, message="class_weight presets.*")

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],   
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

clf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=4, scoring='f1_macro')

grid_search.fit(x_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
Best Score: 0.8152502175138606


In [12]:
random_forest = grid_search.best_estimator_ 

random_forest

RandomForestClassifier(max_features='log2', min_samples_split=10,
                       random_state=42)

### Grid Search for the best Decision Tree Classifier 

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# Define the parameter grid
param_grid = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Create a DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=42)

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=4, scoring='f1_macro')

# Perform grid search
grid_search.fit(x_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 2}
Best Score: 0.7986806079265572


In [14]:
decision_tree = grid_search.best_estimator_

decision_tree

DecisionTreeClassifier(max_features='log2', min_samples_leaf=4, random_state=42)

### Grid Search for the best Stochastic Gradient Descent Classifier

In [15]:
from sklearn.linear_model import SGDClassifier

param_grid = {
    'loss': ['log_loss', 'modified_huber', 'squared_hinge'],
    'penalty': ['l2', 'l1', 'elasticnet'], 
    'alpha': [0.0001, 0.001, 0.01, 0.1], 
    'max_iter': [1000, 3000, 5000],
    'tol': [1e-3, 1e-4, 1e-5],  
}

clf = SGDClassifier(random_state=42)

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=4, scoring='f1_macro')

grid_search.fit(x_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {'alpha': 0.01, 'loss': 'modified_huber', 'max_iter': 1000, 'penalty': 'l2', 'tol': 0.001}
Best Score: 0.7808988943642309


In [16]:
sgd = grid_search.best_estimator_

sgd

SGDClassifier(alpha=0.01, loss='modified_huber', random_state=42)

### Grid Search for the best K-Nearest Neighbors

In [17]:
from sklearn.neighbors import KNeighborsClassifier

param_grid = {
    'n_neighbors': [3, 5, 7, 9],  
    'weights': ['uniform', 'distance'],  
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  
    'p': [1, 2]
}


clf = KNeighborsClassifier()

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=4, scoring='f1_macro')

grid_search.fit(x_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {'algorithm': 'brute', 'n_neighbors': 9, 'p': 2, 'weights': 'uniform'}
Best Score: 0.8013349907519393


In [18]:
knn = grid_search.best_estimator_

knn

KNeighborsClassifier(algorithm='brute', n_neighbors=9)

### Grid Search for the best Multi-layer Perceptron

In [19]:
from sklearn.neural_network import MLPClassifier

param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh', 'logistic'], 
    'solver': ['adam', 'sgd'],  
    'alpha': [0.0001, 0.001, 0.01],  
}



clf = MLPClassifier(random_state=42)

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='f1_macro')

grid_search.fit(x_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'solver': 'adam'}
Best Score: 0.8030504930571951


In [20]:
mlp = grid_search.best_estimator_

mlp

MLPClassifier(hidden_layer_sizes=(50,), random_state=42)

### Performing Ensemble method

In [21]:
from sklearn.ensemble import VotingClassifier

# weighted voting, with proportions 3:2:1:2:2 based on obtained accuracies 
estimators = [
    ('random_forest', random_forest),
    ('decision_tree', decision_tree),
    ('sgd', sgd),
    ('knn', knn),
    ('mlp', mlp),
]

ensemble = VotingClassifier(estimators, voting='soft')

In [22]:
test = data_preparation(test_df)

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       418 non-null    int64  
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      418 non-null    int64  
 6   Embarked  418 non-null    int64  
 7   Title     418 non-null    float64
dtypes: float64(1), int64(6), object(1)
memory usage: 26.2+ KB


In [23]:
x_train = data_preparation(train_df)

x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Pclass    891 non-null    int64 
 1   Sex       891 non-null    object
 2   Age       891 non-null    int64 
 3   SibSp     891 non-null    int64 
 4   Parch     891 non-null    int64 
 5   Fare      891 non-null    int64 
 6   Embarked  891 non-null    int64 
 7   Title     891 non-null    int64 
dtypes: int64(7), object(1)
memory usage: 55.8+ KB


In [24]:
from sklearn.metrics import f1_score, confusion_matrix

ensemble.fit(x_train, y_train)

y_pred = ensemble.predict(test)
y_pred_train = ensemble.predict(x_train)

print(f1_score(y_train, y_pred_train, average='macro'))
confusion_matrix(y_train, y_pred_train)

0.8447971781305115


array([[503,  46],
       [ 82, 260]])

In [25]:
y_pred

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,

In [26]:
def save_predictions(y_pred, name):
    y_pred_series = pd.Series(y_pred, index=test_df['PassengerId'])
    
    y_pred_df = y_pred_series.to_frame().reset_index()

    y_pred_df.rename(columns={0:'Survived'}, inplace=True)

    y_pred_df.to_csv(name, index=False)

### Save results to the CSV file

In [27]:
save_predictions(y_pred, 'result_ensemble.csv')